In [1]:
from huggingface_hub import login
from dotenv import load_dotenv 
import os 

# download .env
load_dotenv("../.env")

try:
    HF_TOKEN= os.getenv('TOKEN_HF')

    if HF_TOKEN is None:
        raise ValueEroor("TOKEN_HF не найдена!")
    
    # login(HF_TOKEN, add_to_git_credential=True)
    login(HF_TOKEN)
except Exception as e:
    print(f"Ошибка: {e}")

In [2]:
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="../data/Vectorbase/VectorStoreIndex_gpt4")
index = load_index_from_storage(storage_context)

In [3]:
from llama_index.core.query_engine import RetrieverQueryEngine

# Создание QueryEngine
query_engine = index.as_query_engine()

In [4]:
#Модель

In [5]:
MODEL_NAME = "IlyaGusev/saiga_llama3_8b"

In [6]:
DEFAULT_SYSTEM_PROMPT = """Ты — MLTeacherBot, интеллектуальный помощник и учитель математики для машинного обучения.
Твоя задача — объяснять сложные математические темы понятным языком, помогать решать задачи и давать полезные советы.

### Твой стиль общения:
- Дружелюбный, терпеливый, мотивирующий.
- Объясняй просто и структурированно, подстраиваясь под уровень пользователя.
- Используй аналогии и примеры.

Ты — наставник, который делает обучение увлекательным и доступным.
"""

In [7]:
import torch
from torch import bfloat16
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig, pipeline
import bitsandbytes


# 4-разрядная конфигурация для загрузки LLM с меньшим объемом памяти графического процессора
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Использую 4-битную квантизацию
    bnb_4bit_quant_type='nf4',  # используем формат NF4
    bnb_4bit_use_double_quant=True,  # применить повторное квантовани
    bnb_4bit_compute_dtype=bfloat16  # Тип из которого преобразуем (доступная точность модели)
)

# Загружаем модель с квантизацией и автоматическим распределением
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto"
)

model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [8]:
# Функция для генерации ответа с использованием базы знаний
def generate_response(query):
    # Поиск релевантной информации в базе знаний
    context = query_engine.query(query)
    
    # Создание промта с контекстом
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    }, {
        "role": "user",
        "content": f"Контекст: {context}\n\nВопрос: {query}"
    }], tokenize=False, add_generation_prompt=True)
    
    # Генерация ответа
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    
    return output

# Пример использования
query = "Что такое математика?"
response = generate_response(query)
print(response)

PermissionDeniedError: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}

In [ ]:
import numpy as np 

# RAG pipeline с FAISS
def rag_pipeline_with_faiss(query):

    query_embedding = np.array([embed_model.get_text_embedding(query)], dtype=np.float32)

    k = 5

    distances, indices = index.search(query_embedding, k)

    retrieved_nodes = [nodes[i] for i in indices[0]]

    context = "\n".join([node.get_text() for node in retrieved_nodes])

    prompt = f"Ответь на вопрос, используя контекст:\n\nКонтекст:\n{context}\n\nВопрос: {query}\n\nОтвет:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=200)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

# Пример использования
query = "как оценивать сосотав совокупности ?"
response = rag_pipeline_with_faiss(query)
print(response)